In [1]:
#Import Dependencies
import pandas as pd

In [4]:
#Read npi chunked datasets
npi2 = pd.read_csv('NEW_NPI_4REAL_2.csv')
npi1 = pd.read_csv('NEW_NPI_4REAL.csv')

In [7]:
#Concat npi chunks
npi = pd.concat([npi1,npi2])

In [12]:
#Clean NPI
#Select desired columns, rename columns, fill na, drop na NPIs, and convert NPI type
npi.columns = [c.upper() for c in npi.columns.values]
npi.columns = [c.replace(' ','_') for c in npi.columns.values]
npi = npi.rename(columns = {'PROVIDER_ORGANIZATION_NAME_LEGAL_BUSINESS_NAME':'PROVIDER_ORGANIZATION_NAME__LEGAL_BUSINESS_NAME'})
column_string = 'NPI,PROVIDER_ORGANIZATION_NAME_(LEGAL_BUSINESS_NAME),Healthcare_Provider_Taxonomy_Code_1,Healthcare_Provider_Taxonomy_Code_2,Healthcare_Provider_Taxonomy_Code_3,Healthcare_Provider_Taxonomy_Code_4,Healthcare_Provider_Taxonomy_Code_5,Healthcare_Provider_Taxonomy_Code_6,Healthcare_Provider_Taxonomy_Code_7,Healthcare_Provider_Taxonomy_Code_8,Healthcare_Provider_Taxonomy_Code_9,Healthcare_Provider_Taxonomy_Code_10,Healthcare_Provider_Taxonomy_Code_11,Healthcare_Provider_Taxonomy_Code_12,Healthcare_Provider_Taxonomy_Code_13,Healthcare_Provider_Taxonomy_Code_14,Healthcare_Provider_Taxonomy_Code_15,Provider_Other_Organization_Name,Provider_First_Line_Business_Mailing_Address,Provider_Second_Line_Business_Mailing_Address,Provider_Business_Mailing_Address_City_Name,Provider_Business_Mailing_Address_State_Name,Provider_Business_Mailing_Address_Postal_Code,Provider_Business_Mailing_Address_Telephone_Number,Provider_First_Line_Business_Practice_Location_Address,Provider_Second_Line_Business_Practice_Location_Address,Provider_Business_Practice_Location_Address_City_Name,Provider_Business_Practice_Location_Address_State_Name,Provider_Business_Practice_Location_Address_Postal_Code,Provider_Business_Practice_Location_Address_Telephone_Number,Last_Update_Date'
NPI = npi[column_string.upper().split(',')]
NPI =NPI.rename(columns ={'PROVIDER_ORGANIZATION_NAME_(LEGAL_BUSINESS_NAME)':'NAME',
 'HEALTHCARE_PROVIDER_TAXONOMY_CODE_1':'CODE_1',
 'HEALTHCARE_PROVIDER_TAXONOMY_CODE_2':'CODE_2',
 'HEALTHCARE_PROVIDER_TAXONOMY_CODE_3':'CODE_3',
 'HEALTHCARE_PROVIDER_TAXONOMY_CODE_4':'CODE_4',
 'HEALTHCARE_PROVIDER_TAXONOMY_CODE_5':'CODE_5',
 'HEALTHCARE_PROVIDER_TAXONOMY_CODE_6':'CODE_6',
 'HEALTHCARE_PROVIDER_TAXONOMY_CODE_7':'CODE_7',
 'HEALTHCARE_PROVIDER_TAXONOMY_CODE_8':'CODE_8',
 'HEALTHCARE_PROVIDER_TAXONOMY_CODE_9':'CODE_9',
 'HEALTHCARE_PROVIDER_TAXONOMY_CODE_10':'CODE_10',
 'HEALTHCARE_PROVIDER_TAXONOMY_CODE_11':'CODE_11',
 'HEALTHCARE_PROVIDER_TAXONOMY_CODE_12':'CODE_12',
 'HEALTHCARE_PROVIDER_TAXONOMY_CODE_13':'CODE_13',
 'HEALTHCARE_PROVIDER_TAXONOMY_CODE_14':'CODE_14',
 'HEALTHCARE_PROVIDER_TAXONOMY_CODE_15':'CODE_15',
 'PROVIDER_OTHER_ORGANIZATION_NAME':'OTHER_NAME',
 'PROVIDER_FIRST_LINE_BUSINESS_MAILING_ADDRESS':'MAILING_ADDRESS_1',
 'PROVIDER_SECOND_LINE_BUSINESS_MAILING_ADDRESS':'MAILING_ADDRESS_2',
 'PROVIDER_BUSINESS_MAILING_ADDRESS_CITY_NAME':'MAILING_CITY',
 'PROVIDER_BUSINESS_MAILING_ADDRESS_STATE_NAME':'MAILING_STATE',
 'PROVIDER_BUSINESS_MAILING_ADDRESS_POSTAL_CODE':"MAIING_ZIP",
 'PROVIDER_BUSINESS_MAILING_ADDRESS_TELEPHONE_NUMBER':'MAILING_PHONE',
 'PROVIDER_FIRST_LINE_BUSINESS_PRACTICE_LOCATION_ADDRESS':'LOCATION_ADDRESS_1',
 'PROVIDER_SECOND_LINE_BUSINESS_PRACTICE_LOCATION_ADDRESS':'LOCATION_ADDRESS_2',
 'PROVIDER_BUSINESS_PRACTICE_LOCATION_ADDRESS_CITY_NAME':'LOCATION_CITY',
 'PROVIDER_BUSINESS_PRACTICE_LOCATION_ADDRESS_STATE_NAME':'LOCATION_STATE',
 'PROVIDER_BUSINESS_PRACTICE_LOCATION_ADDRESS_POSTAL_CODE':'LOCATION_ZIP',
 'PROVIDER_BUSINESS_PRACTICE_LOCATION_ADDRESS_TELEPHONE_NUMBER':'LOCATION_PHONE'})
NPI = NPI.fillna('None')
NPI = NPI[NPI.NPI!='None']
NPI.NPI = NPI.NPI.astype('int').astype('str')

In [13]:
#Put all taxonomies into one column
dit_list = []
for row in NPI.itertuples():
    group_list = [row.CODE_1,row.CODE_2,row.CODE_3,row.CODE_4,row.CODE_5,row.CODE_6,row.CODE_7,row.CODE_8,row.CODE_9,row.CODE_10,row.CODE_11,row.CODE_12,row.CODE_13,row.CODE_14,row.CODE_15]
    final_code = 'None'
    final_code_list = []
    for code in group_list:
        if code != 'None':
            if code != final_code:
                final_code = code
                final_code_list.append(code)
    if final_code != 'None':
        dit ={}
        dit['NPI']=row.NPI
        if 'None' in final_code_list:
            final_code_list.remove('None')
        dit['Taxonomies'] = final_code_list
        dit_list.append(dit)
taxonomy_scrunch = pd.DataFrame(dit_list)
taxonomy_scrunch.NPI = taxonomy_scrunch.NPI.astype('int').astype('str')

In [14]:
#Dop more unnecessary columns
npi_with_taxes = pd.merge(taxonomy_scrunch, NPI, on='NPI').drop(columns=['CODE_1', 'CODE_2', 'CODE_3', 'CODE_4', 'CODE_5', 'CODE_6',
       'CODE_7', 'CODE_8', 'CODE_9', 'CODE_10', 'CODE_11', 'CODE_12',
       'CODE_13', 'CODE_14', 'CODE_15'])

In [15]:
#Hospital taxonomy list
actual_hospital_tax_codes = ['287300000X',
'281P00000X',
'281PC2000X',
'282N00000X',
'282NC2000X',
'282NC0060X',
'282NR1301X',
'282NW0100X',
'282E00000X',
'286500000X',
'2865C1500X',
'2865M2000X',
'2865X1600X',
'283Q00000X',
'283X00000X',
'283XC2000X',
'282J00000X',
'284300000X']

In [16]:
#Find hospitals
names= []
hospital_npis = []
for row in npi_with_taxes.itertuples():
    is_hospital = False
    for tax_code in actual_hospital_tax_codes:
        if tax_code in row.Taxonomies:
            is_hospital = True
    if is_hospital == True:
        hospital_npis.append(row.NPI)
        print(row.NAME)
        names.append(row.NAME)

CUMBERLAND COUNTY HOSPITAL SYSTEM, INC
CUMBERLAND COUNTY HOSPITAL SYSTEM, INC
CUMBERLAND COUNTY HOSPITAL SYSTEM, INC
CUMBERLAND COUNTY HOSPITAL SYSTEM, INC
ROBERT WOOD JOHNSON UNIVERSITY HOSPITAL, INC
ROBERT WOOD JOHNSON UNIVERSITY HOSPITAL, INC
ROBERT WOOD JOHNSON UNIVERSITY HOSPITAL, INC
ROBERT WOOD JOHNSON UNIVERSITY HOSPITAL, INC
MONTICELLO BIG LAKE HOSPITAL NURSING HOME
MONTICELLO BIG LAKE HOSPITAL NURSING HOME
MONTICELLO BIG LAKE HOSPITAL NURSING HOME
MONTICELLO BIG LAKE HOSPITAL NURSING HOME
EDEN MEDICAL CENTER
EDEN MEDICAL CENTER
EDEN MEDICAL CENTER
EDEN MEDICAL CENTER
LINDSBORG COMMUNITY HOSPITAL ASSOCIATION
LINDSBORG COMMUNITY HOSPITAL ASSOCIATION
LINDSBORG COMMUNITY HOSPITAL ASSOCIATION
LINDSBORG COMMUNITY HOSPITAL ASSOCIATION
BROADVIEW, INC.
BROADVIEW, INC.
BROADVIEW, INC.
BROADVIEW, INC.
ST LUKES REGIONAL MEDICAL CENTER
ST LUKES REGIONAL MEDICAL CENTER
ST LUKES REGIONAL MEDICAL CENTER
ST LUKES REGIONAL MEDICAL CENTER
OKLAHOMA HEART HOSPITAL, LLC
OKLAHOMA HEART HOSPITAL, LL

ST. JOSEPH'S HOSPITAL
LEELANAU MEMORIAL HEALTH CENTER
PAUL OLIVER MEMORIAL HOSPITAL
SELECT SPECIALTY HOSPITAL - ANN ARBOR INC
UPPER CHESAPEAKE MEDICAL CENTER, INC.
AMERICAN TRANSITIONAL HOSPITALS LLC
JAY COUNTY HOSPITAL
HARFORD MEMORIAL HOSPITAL
SELECT SPECIALTY HOSPITAL - AKRON LLC
VIRTUA WEST JERSEY HEALTH SYSTEM INC
KALKASKA MEMORIAL HEALTH CENTER
LIGA PUERTORRIQUENA CONTRA EL CANCER
NORTON COUNTY HOSPITAL
ASHLEY MEDICAL CENTER
VIRTUA MEMORIAL HOSPITAL OF BURLINGTON COUNTY, INC.
SELECT SPECIALTY HOSPITAL - ARIZONA INC
NORMAN REGIONAL HOSPITAL AUTHORITY
STONEWALL JACKSON MEMORIAL HOSPITAL COMPANY
MUNSON MEDICAL CENTER
AMERICAN TRANSITIONAL HOSPITALS, INC.
FOUR WINDS OF SARATOGA
ADVENTIST HEALTHCARE, INC.
NORTH HAWAII COMMUNITY HOSPITAL,INC
DECATUR HEALTH SYSTEMS INC
FOUR WINDS, INC.
ADVENTIST HEALTHCARE, INC.
ADVENTIST HEALTHCARE, INC.
EASTERN PLUMAS HEALTH CARE DISTRICT
LAURENS COUNTY HEALTH CARE SYSTEM
LISBON AREA HEALTH SERVICES
NATHAN LITTAUER HOSPITAL ASSOCIATION
UNIVERSITY HOSP

FREEMAN REGIONAL HEALTH SERVICES
BRFHH SHREVEPORT LLC
SOUTHERN REGIONAL HEALTH SYSTEM, INC
BRFHH MONROE LLC
REGENCY HOSPITAL COMPANY OF SOUTH CAROLINA, LLC
REGENCY HOSPITAL COMPANY OF MERIDIAN, LLC
REGENCY HOSPITAL COMPANY OF SOUTH ATLANTA, LLC
SCHUYLKILL MEDICAL CENTER - EAST NORWEIGAN STREET
REGENCY HOSPITAL OF ODESSA LLLP
AHS HOSPITAL CORP
ST. MARY'S WARRICK HOSPITAL, INC
REGENCY HOSPITAL OF AKRON, LLC
REGENCY HOSPITAL OF NORTHWEST INDIANA, LLC
SAINT MICHAEL'S MEDICAL CENTER, INC.
CATHEDRAL HEALTH SERVICES, INC
COLUMBUS HOSPITAL
MONSOUR MEDICAL CENTER
HANSFORD COUNTY HOSPITAL DISTRICT
ST. JOSEPH MEDICAL CENTER
ST. MARY'S MEDICAL CENTER
YAMPA VALLEY MEDICAL CENTER
COMMUNITY MEMORIAL HOSPITAL
ST ELIZABETH ANN SETON HOSPITAL
SOUTH BIG HORN COUNTY HOSPITAL DISTRICT
LITTLE RIVER MEDICAL CENTER, INC.
REGENCY HOSPITAL OF HATTIESBURG, LLC
REGENCY HOSPITAL OF NORTHWEST ARKANSAS, LLC
COVINGTON -AMG SPECIALTY HOSPITAL, LLC
REGENCY HOSPITAL COMPANY OF MACON LLC
REGENCY HOSPITAL OF GREENVILLE, L

NORTHERN COLORADO REHABILITATION HOSPITAL INC
ST.ELIZABETH-SKILLED FACILITY
PEKIN MEMORIAL HOSPITAL
MARTIN COUNTY HOSPITAL DISTRICT
HUNTERDON MEDICAL CENTER
DAVIS MEMORIAL HOSPITAL
RANDOLPH HOSPITAL INC.
WINDSOR HOSPITAL CORPORATION
DAKOTA SPECIALTY INSTITUTE
WINDSOR HOSPITAL CORPORATION
TOOLE COUNTY
WINDSOR HOSPITAL CORPORATION
TANNER MEDICAL CENTER, INC
TMC/VILLA RICA HOSPITAL, INC
TMC/HIGGINS GENERAL HOSPITAL
COMANCHE COUNTY CONSOLIDATED HOSPITAL DISTRICT
JOHNSON REGIONAL MEDICAL CENTER
POCONO MEDICAL CENTER, INC.
TRANSYLVANIA COMMUNITY HOSPITAL, INC
ILLINOIS DEPARTMENT OF HUMAN SERVICES
ST FRANCIS MEDICAL CENTER
LEVINDALE HEBREW GERIATRIC CENTER AND HOSPITAL
WEST BRANCH REGIONAL MEDICAL CENTER
FIRELANDS REGIONAL MEDICAL CENTER
INTRACARE HOSPITAL NORTH
INTRACARE HOSPITAL
WINDOM AREA HOSPITAL
HCAP VENTURE GROUP, LLC
IREDELL MEMORIAL HOSPITAL, INCORPORATED
EAST CAROLINA HEALTH - CHOWAN INC
AMERIS OF ARKANSAS LLC
LINCOLN HEALTH SYSTEM
INTEGRATED OF AMARILLO INC
CROUSE HEALTH HOSPITAL, 

NORTH OAKS REHABILITATION HOSPITAL, LLC
EAST CAROLINA HEALTH - HERITAGE INC
MEDINA HOSPITAL
WITHAM MEMORIAL HOSPITAL
SALINE HOSPITAL LLC
CAMPBELL COUNTY HMA, LLC
PHC-ASHLAND, L.P.
EAST CAROLINA HEALTH-BERTIE
THREE RIVERS HEALTH AUTHORITY
GRANT COUNTY PUBLIC HOSPITAL DISTRICT 3
UNITED HOSPITAL CENTER, INC
MERCY HEALTH SYSTEM, INC.
JEFFERSON COUNTY HMA LLC
HOSPITAL FOR SICK CHILDREN
BROWN COUNTY HOSPITAL
NORTHWEST MEDICAL FOUNDATION OF TILLAMOOK
TWIN CITY HOSPITAL CORPORATION
GOTTLIEB COMMUNITY HEALTH SERVICES CORPORATION
COPPER BASIN COMMUNITY HOSPITAL INC
FAULKTON AREA MEDICAL CENTER
PENNSYLVANIA DEPARTMENT OF PUBLIC WELFARE
KIOWA COUNTY HOSPITAL AUTHORITY
SHARP CORONADO HOSPITAL AND HEALTHCARE CENTER
ST.JOHN'S PLEASANT VALLEY HOSPITAL
SOUTH COAST MEDICAL CENTER
WOMEN & INFANTS HOSPITAL OF RHODE ISLAND
PAM II OF COVINGTON, LLC
CUBA MEMORIAL HOSPITAL, INC.
SHARP CHULA VISTA MEDICAL CENTER
OSAWATOMIE STATE HOSPITAL
GROSSMONT HOSPITAL CORPORATION
CRAWFORD LONG HOSPITAL
DEWITT HOSPITAL & N

...

ASPIRUS RIVERVIEW HOSPITAL & CLINICS, INC.
SELAH HEALTH, LLC
STANFORD HEALTH CARE
UNIVERSITY OF CALIFORNIA SAN FRANCISCO
MEDICA SAN MIGUEL
CHILDRENS NATIONAL HOSPITAL
CLINIQUE SPECIALISEE DU CAP-HAITIEN
COLUSA MEDICAL CENTER LLC
PATIENT HOUSING AT ANMC
WOMEN'S CENTER FOR PELVIC HEALTH
OHIOHEALTH
SAINT MARY'S PASSAIC LLC
THE WALTER B. JONES CENTER WOODSIDE TREATMENT CENTER
LOVELACE UNM REHABILITATION HOSPITAL LLC
KERN MEDICAL
SAINT MICHAEL'S MEDICAL CENTER
NAPA STATE HOSPITALS
GEORGIA PLASTIC SURGERY SPECIALIST
MIAMI VA HEALTHCARE SYSTEM
JANICE HOLLIER M.D.
WALTER B JONES CENTER LAKESIDE PSYCHIATRIC HOSPITAL
NORTHWELL LONG ISLAND JEWISH HOSPITAL
COLUSA MEDICAL CENTER LLC
VICTORIA MEDICAL CANCUN SA DE CV
TULSA MICRO HOSPITAL, LLC
MEMORIAL SLOAN KETTERING CANCER CENTER
BROOKDALE HOSPITAL
MCHS HOSPITALS, INC.
CENTRAL CAROLINA HOSPITAL
MEMORIAL HOSPITAL
CONTINUECARE HOSPITAL AT PALMETTO HEALTH BAPTIST, INC.
BAPTIST HOSPITAL OF SOUTHEAST TEXAS
PHOENIX VA HEALTHCARE SYSTEM
KD SERVICES
PENN PR

NATCHAUG HOSPITAL
ADAMS COUNTY HOSPITAL DISTRICT 2
ALEXANDER TRAN MD PA
LHCG CXIX, LLC
LHCG CXVIII, LLC
DAYTON CHILDREN'S HOSPITAL
CLINICAL SOLUTIONS LLC
VARIETY CHILDREN'S HOSPITAL
TANNER MEDICAL CENTER ALABAMA INC
PACIFIC GARDENS MEDICAL CENTER, LLC
LHCG CXX, LLC
HCN EP HORIZON CITY, LLC
UPMC SUSQUEHANNA SUNBURY
HCN EP HORIZON CITY, LLC
TRINITAS REGIONAL MEDICAL CENTER
UPMC SUSQUEHANNA LOCK HAVEN
LOMA LINDA UNIVERSITY BEHAVIORAL MEDICINE CENTER, INC.
BLUE NET NUEVA INVERSION SA DE CV
NORTHEASTERN VERMONT REGIONAL HOSPITAL, INC
INTEGRA HEALTH SERVICES PROFESSIONAL CORPORATION
OU MEDICINE INC.
WEATHERFORD HEALTH SERVICES, LLC
BRANDYWINE HOSPITAL, LLC
PHOENIXVILLE HOSPITAL, LLC
JENNERSVILLE HOSPITAL, LLC
POTTSTOWN HOSPITAL, LLC
CHESTNUT HILL HOSPITAL, LLC
LHCG CXXII, LLC
LHCG CXXI, LLC
NORTH OAK REGIONAL HOSPITAL, INC
NORTH OAK REGIONAL HOSPITAL INC
DREW HO MD, INC
MULTICARE HEALTH SYSTEM
SHC MEDICAL CENTER - TOPPENISH
SHC MEDICAL CENTER - YAKIMA
SAGE LTAC LLC
FRANCISCAN SPECIALTY CARE,

PAM SPECIALTY HOSPITAL OF LUFKIN, LLC
VIBRA REHABILITATION HOSPITAL OF SOUTHERN INDIANA, LLC
MID COAST HOSPITAL
UKHS GREAT BEND, LLC
WELLCARE HEALTH SYSTEM, LLC
BAPTIST MEMORIAL HOSPITAL
RUSHIN PATEL MD LLC
SUMACA S.A DE C.V.
PERRY COUNTY HEALTH SYSTEM
WOODLANDS INTEGRATIVE CARE HOSPITAL
BELLIN HEALTH IRON MOUNTAIN INC
SAN ANTONIO-AMG SPECIALTY HOSPITAL, LLC
BENSON HOSPITAL CORPORATION
CENTERPOINTE HOSPITAL OF COLUMBIA, LLC
JAVON BEA HOSPITAL
RAYS OF LIGHT COMPANION CARE AGENCY LLC
CENTRAL PARK WEST CARE LLC
FLORIDA HOSPITAL OCALA INC
ST MARTIN HOSPITAL INC
HOSPITAL SERVICE DISTRICT 1 OF EAST BATON ROUGE PARISH DBA LANE REGION
FARHAD NAJAFI, MD INC.
EDGEWATER SYSTEMS FOR BALANCED LIVING, INC
ARH MARY BRECKINRIDGE HEALTH SERVICES, INC.
SRP SAYRE, LLC
CALDWELL MEMORIAL HOSPITAL INC
ABSOLUTE CARE, LLC
LANDMARK REHABILITATION HOSPITAL OF CAPE GIRARDEAU, LLC
GEORGE RYAN GILLIGAN MD PA
DESTINY SPRINGS HEALTHCARE, LLC
LOURDES HOSPITAL LLC
NOVANT HEALTH REHABILITATION HOSPITAL OF WINSTON-SALEM

VANDERBILT UNIVERSITY MEDICAL CENTER
SURGICAL ASSISTING OF SOUTHERN NEW JERSEY
NICHOLAS LAPARA III MD, LLC
KPC PROMISE HOSPITAL OF WICHITA FALLS LLC
KPC PROMISE HOSPITAL OF BATON ROUGE LLC
MISSIONARIES OF JESUS THE SAVIOR (MJS)
INTENSIVE SPECIALTY HOSPITAL, LLC
KPC PROMISE HOSPITAL OF SALT LAKE LLC
KPC PROMISE HOSPITAL OF OVERLAND PARK LLC
KPC PROMISE HOSPITAL OF PHOENIX LLC
KPC PROMISE HOSPITAL OF VICKSBURG LLC
KPC PROMISE HOSPITAL OF DALLAS LLC
MENTOR ABI, LLC
PERIMETER BEHAVIORAL HOSPITAL OF GARLAND, LLC
PERIMETER BEHAVIORAL HOSPITAL OF ARLINGTON, LLC
HANANIAH CHRISTIAN COUNSELING
ALL ABOUTMEDTRANS
PRIVATE HEALTHCARE FACILITIES
TRIHEALTH HOSPITAL , INC
POUDRE VALLEY HEALTH CARE INC
COALINGA MEDICAL CENTER, LLC
LINCOLN MEDICAL AND MENTAL HEALTH CENTER
UNIVERSAL BEHAVIORAL HEALTH HOSPITAL INC
DOCTORS HOSPITAL OF SARASOTA
GRACE OUTREACH MINISTRIES INTERNATIONAL, INC
HOSPITAL WOOD MEDICAL CENTER
ETHICUS HOSPITAL DFW LLC
NYU LANGONE HOSPITALS
NORTHSHORE EXTENDED CARE HOSPITAL, LLC
MILLAR

JOSHUA MORRIS
DUBLIN REHABILITATION HOSPITAL, LLC
INDIANAPOLIS REHABILITATION HOSPITAL, LLC
DENVER REHABILITATION HOSPITAL, LLC
TEMPLE UNIVERSITY HOSPITAL, INC
JOAQUIN BERMUDEZ PC
SOUTHERN BAPTIST HOSPITAL OF FLORIDA INC
NEUROHEALTH SOLUTIONS PLLC
REHABILITATION HOSPITAL OF NORTHERN INDIANA, LLC
EVEREST REHABILITATION HOSPITAL KELLER, LLC
PUERTO RICO ADVANCED UROLOGY CENTER, LLC
LEHIGH VALLEY HOSPITAL - COORDINATED HEALTH ALLENTOWN
PORTERCARE ADVENTIST HEALTH SYSTEM
CATHOLIC HEALTH INITIATIVES COLORADO
CATHOLIC HEALTH INITIATIVES COLORADO
CATHOLIC HEALTH INITIATIVES COLORADO
CATHOLIC HEALTH INITIATIVES COLORADO
PORTERCARE ADVENTIST HEALTH SYSTEM
CATHOLIC HEALTH INITIATIVES COLORADO
PORTERCARE ADVENTIST HEALTH SYSTEM
PORTERCARE ADVENTIST HEALTH SYSTEM
CATHOLIC HEALTH INITIATIVES COLORADO


In [17]:
#Make hospital df
npi_hospital_df = npi_with_taxes[npi_with_taxes['NPI'].isin(hospital_npis)].sort_values('NAME')

In [19]:
len(npi_hospital_df)

25249

In [21]:
#Save
npi_hospital_df.to_csv('NPI_DATA/NPI_Hospitals_20-01-12.csv',index=False)